In [22]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [23]:
ori_data = np.load('../Data/samples_24/sine_ground_truth.npy')
fake_data = np.load('../Data/ldm_fake_sine_10_1_16_2.npy')
iterations = 5

In [24]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.003991649324298281 

Iter 1:  context-fid = 0.0037883193726177206 

Iter 2:  context-fid = 0.0033507767815540617 

Iter 3:  context-fid = 0.00398663866752831 

Iter 4:  context-fid = 0.004587152832187101 

Final Score:  0.003940907395637095 ± 0.0005531458087384559


In [25]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.018125241159849686 

Iter 1:  cross-correlation = 0.009053208754259861 

Iter 2:  cross-correlation = 0.013390322353697193 

Iter 3:  cross-correlation = 0.017719813824989168 

Iter 4:  cross-correlation = 0.009176767061712004 

Final Score:  0.013493070630901582 ± 0.005471631917036357


In [26]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [27]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 0:  0.0047499999999999765 , 0.452 , 0.5385 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 1:  0.009749999999999981 , 0.4825 , 0.498 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 2:  0.0 , 0.4925 , 0.5075 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 3:  0.005249999999999977 , 0.465 , 0.5455 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 4:  0.00824999999999998 , 0.491 , 0.4925 

sine:
Final Score:  0.005599999999999983 ± 0.004664510408295171



In [28]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training:   0%|          | 0/5000 [00:00<?, ?it/s]

0  epoch:  0.09296893159800253 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

1  epoch:  0.09314948075242335 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

2  epoch:  0.09336988887428098 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

3  epoch:  0.09285637082141068 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

4  epoch:  0.09325685030921844 

sine:
Final Score:  0.0931203044710672 ± 0.0002592396350288922

